In [ ]:
# Import necessary libraries
import gzip
import pandas as pd
import re
import matplotlib.pyplot as plt

In [ ]:
# Download the data using wget and unzip the file
!wget https://ftp.ebi.ac.uk/pub/databases/spot/pgs/scores/PGS001298/ScoringFiles/Harmonized/PGS001298_hmPOS_GRCh38.txt.gz
!gunzip PGS001298_hmPOS_GRCh38.txt.gz

In [ ]:
# Load the file, ignoring comment lines starting with "#"
df = pd.read_csv(
    "PGS001298_hmPOS_GRCh38.txt",
    sep="\t",
    comment="#"   # load all as string to avoid dtype errors
)

print("Initial shape:", df.shape)
df.head()



In [ ]:
# Extract clean chromosome numbers from entries like:
# "21", "chr21", "21_KI270...", etc.

def clean_chr(value):
    if pd.isna(value):
        return None
    match = re.match(r"^(?:chr)?(\d+)", str(value))
    return match.group(1) if match else None

df["hm_chr_clean"] = df["hm_chr"].apply(clean_chr)

print(df["hm_chr_clean"].unique()[:22])


In [ ]:
# Remove rows with missing values
df_clean = df.dropna(subset=["hm_chr_clean", "hm_pos", "effect_weight"])
print("Cleaned shape:", df_clean.shape)

In [ ]:
# Exploratory Analysis
print("Number of variants:", len(df_clean))
print("\nColumns available:", list(df_clean.columns))

print("\nSummary statistics for effect_weight:")
print(df_clean["effect_weight"].describe())

print("Skewness:", df_clean['effect_weight'].skew())
print("Kurtosis:", df_clean['effect_weight'].kurt())



In [ ]:
# Filter Chromosome 21 Variants
chr21_df = df_clean[df_clean["hm_chr_clean"] == "21"]

print("Variants mapped to chromosome 21:", len(chr21_df))
chr21_df.head()

In [ ]:
# Histogram of Chromosome 21
plt.figure(figsize=(8,5))
plt.hist(chr21_df["effect_weight"], bins=30)
plt.xlabel("Effect Weight")
plt.ylabel("Frequency")
plt.title("Distribution of Effect Weights – Chromosome 21 (PGS001298)")
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
print("""
CONCLUSION:

1. Most effect weights are centered around zero, indicating very small individual contributions of chromosome 21 variants to the overall polygenic score.
2. The histogram displays a roughly symmetric, bell-shaped distribution, suggesting no strong skew toward positive or negative effects.
3. A majority of the variants fall within the range –0.005 to +0.005, showing that effect sizes are generally minimal.
4. A small number of variants exhibit slightly larger positive or negative effects (approximately –0.03 to +0.02), but none represent large or dominant influences.
5. The overall pattern is consistent with polygenic architecture, where many small-effect variants collectively contribute to trait risk rather than a few high-impact loci.

These observations confirm that chromosome 21 supports polygenic score PGS001298.
""")
